# Car Price prediction

**Цель проекта** - предсказать цену автомобиля

**Задачи:** 
 - Написать программу по сбору данных об автомобилях с сайта auto.ru и выгрузить эти данные
 - Подготовить набор данных для обучения модели
 - Потренировать работу с pandas на реальных данных
 - Попрактиковаться в предобработке различных данных, в частности с пропущенными данными (Nan) и с различными видами кодирования признаков
 - Попрактиковаться в Feature Engineering
 - Попрактиковаться в построении различных моделях, в частности ансамбля алгоритмов


# 1. Import

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import re
import math
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, cohen_kappa_score

from datetime import datetime, timedelta
from tqdm.notebook import tqdm

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor

from itertools import combinations
from scipy.stats import ttest_ind

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Уберем Warnings из отображения
import warnings; warnings.simplefilter('ignore')

# Поправим отображение дасетов
pd.set_option('display.max_columns', None)

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы
!pip freeze > requirements.txt

# зафиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы
RANDOM_SEED = 42

**МЕТРИКА**

Определим метрику для проверки моделей - MAPE  (Mean Percentage Absolute Error, расшифровывается выражение как средняя абсолютная ошибка в процентах).

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# 2. Setup

In [ ]:
VERSION    = 19 # версия запуска для сохранения ответов

DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-10072021/'
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
FILE_TRAIN = 'all_auto_ru_10_07_2021.csv'
FILE_TEST  = 'test.csv'
FILE_SUB   =  'sample_submission.csv'

VAL_SIZE   = 0.20   # 20% для разделения выборки на обучающую и тестовую

# 3. Data

In [ ]:
def load_data_frames():
    # Выгружает три датасета
    train = pd.read_csv(DIR_TRAIN + FILE_TRAIN)
    test = pd.read_csv(DIR_TEST + FILE_TEST)
    sample_submission = pd.read_csv(DIR_TEST + FILE_SUB)
    
    return train, test, sample_submission

In [ ]:
train, test, sample_submission = load_data_frames()

In [ ]:
train.sample(2)

In [ ]:
test.sample(2)

In [ ]:
train.info()

In [ ]:
test.info()

Форматы данных в обучающем и тестовом датасетах немного отлючаются. Для того, чтобы объединить их приведем форматы к одному виду.

In [ ]:
def rename_columns_train(train: pd.DataFrame) -> None: 
    # Переименуем названия признаков в train датасете
    train.columns = [
        'body_type', 'brand', 'color', 'fuel_type', 'model_date', 'name',
        'number_of_doors', 'production_date', 'vehicle_transmission',
        'engine_displacement', 'vehicle_configuration', 'engine_power',
        'description', 'mileage', 'drive_type', 'wheel', 'owners', 'pts',
        'customs', 'ownership', 'price', 'model_name', 'vendor', 'equipment_dict',
        'complectation_dict'
    ]

In [ ]:
def rename_columns_test(test: pd.DataFrame) -> None: 
    # Переименуем названия признаков в test датасете
    test.columns = [
        'body_type', 'brand', 'car_url', 'color', 'complectation_dict',
        'description', 'engine_displacement', 'engine_power', 'equipment_dict',
        'fuel_type', 'image', 'mileage', 'model_date', 'model_info', 'model_name',
        'name', 'number_of_doors', 'parsing_unixtime', 'price_currency',
        'production_date', 'sell_id', 'super_gen', 'vehicle_configuration',
        'vehicle_transmission', 'vendor', 'owners', 'ownership', 'pts',
        'drive_type', 'wheel', 'condition', 'customs'
    ]

In [ ]:
# engineDisplacement

def search_engine_displacement(text: str) -> float:
    pattern = '\d\.\d'
    match = re.findall(pattern, str(text))
    n = len(match)
    
    if n == 2:
        return float(match[1])
    elif n == 1:
        return float(match[0])
    else:
        return .0

In [ ]:
def prepare_columns_format_train(train: pd.DataFrame) -> None: 
    # 1. Поле color в train представлено в виде hex_code
    color_hex_dict = {
        '040001': 'чёрный', 'FAFBFB': 'белый', '97948F': 'серый', '0000CC': 'синий', 
        '007F00': 'зелёный', '200204': 'красный', 'CACECB': 'серебристый', '660099': 'фиолетовый', 
        'C49648': 'оранжевый', '22A0F8': 'голубой', 'DEA522': 'оранжевый', '4A2197': 'фиолетовый', 
        'FF8649': 'оранжевый', 'FFD600': 'жёлтый', 'FFC0CB': 'розовый', 'EE1D19': 'красный'
    }
    train.loc[:, 'color'] = train['color'].apply(lambda x: color_hex_dict[x])
    
    # 2. Владельцы меняем в train
    train.loc[:, 'owners'] = train['owners'].apply(lambda x: 3 if x > 3 else x)
    
    # 3. engineDisplacement
    train.loc[:, 'engine_displacement'] = train['name'].apply(search_engine_displacement)
    
    # 4. время парсинга 2021-07-10 02:07:00 - 1625882847
    train.loc[:, 'parsing_unixtime'] = 1625882847

In [ ]:
def prepare_columns_format_test(test:  pd.DataFrame) -> None:
    # 1. Поле engine_power в test представлено в виде "306 N12", нам надо только число Л.С.
    test.loc[:, 'engine_power'] = test['engine_power'].apply(lambda x: str(x).split(' ')[0])
    
    # 2. wheel(Руль) 
    wheel_dict = {'Левый': 'LEFT', 'Правый': 'RIGHT'}
    test.loc[:, 'wheel'] = test['wheel'].apply(lambda x: wheel_dict[x])
    
    # 3. ПТС
    pts_dict = {'Оригинал': 'ORIGINAL', 'Дубликат': 'DUPLICATE'}
    test.loc[:, 'pts'] = test['pts'].apply(lambda x: pts_dict[x] if x == x else None)
    
    # 4. Владельцы меняем в test
    test.loc[:, 'owners'] = test['owners'].apply(lambda x: float(x[:1]))
    
    # 5. vehicleTransmission меняем в test
    vehicle_transmission_dict = {
        'автоматическая': 'AUTOMATIC', 'роботизированная': 'ROBOT', 
        'механическая': 'MECHANICAL', 'вариатор': 'VARIATOR'
    }
    test.loc[:, 'vehicle_transmission'] = test['vehicle_transmission'].apply(lambda x: vehicle_transmission_dict[x])
    
    # 6. engineDisplacement
    test.loc[:, 'engine_displacement'] = test['engine_displacement'].apply(search_engine_displacement)

In [ ]:
def concatenate_data_frames(train: pd.DataFrame, test: pd.DataFrame) -> pd.DataFrame:
    # ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
    columns = [
        'body_type', 'brand', 'color', 'fuel_type', 'mileage', 'model_date', 
        'name', 'number_of_doors', 'engine_power', 'wheel', 'drive_type', 'pts', 
        'owners', 'vehicle_transmission', 'production_date', 'vendor',
        'model_name', 'engine_displacement', 'ownership', 'complectation_dict', 
        'equipment_dict', 'parsing_unixtime'
    ]
    # 'ownership' - надо вытаскивать данные (количество месяцев владения)
    # 'complectation_dict' - надо вытаскивать данные
    # 'equipment_dict' - надо вытаскивать данные
    
    df_train = train[columns]
    df_test = test[columns]

    df_train['price'] = train['price']  # Целевая переменная трейна
    df_test['price'] = 0                # Целевой переменной теста нет, поэтому занулим пока
    
    df_train['sample'] = 1 # помечаем где у нас трейн
    df_test['sample'] = 0  # помечаем где у нас тест

    return df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем


In [ ]:
# Загрузим данные
train, test, sample_submission = load_data_frames()

# Согласуем наименования признаков
rename_columns_train(train)
rename_columns_test(test)

# Согласуем форматы признаков
prepare_columns_format_train(train)
prepare_columns_format_test(test)

# Объединим датасеты в один
data = concatenate_data_frames(train, test)

In [ ]:
data.sample(5)

# 4. Cleaning and Prepping Data

## Определим вспомогательные функции

In [ ]:
def data_full_info(data, full=True, short=False):
    '''Функция для вывода общей информации по датасету.
    data - набор исходных данных
    full - флаг вывода информации для количественных признаков
    short - флаг вывода информации из первых трех строк датасета
    
    Функция для выводит общую информацию по датасету.
    
    Если установлен флаг short, то отобразить первые три строки из датасета.
    В ином случае - не отображать.
    
    
    Если установлен флаг full, то отобразить информацию для количественных признаков.
    В ином случае - не отображать.
    '''
        
    list_of_names = list(data.columns)
    temp_dict = {}
    temp_dict['имя признака'] = list_of_names
    temp_dict['тип'] = data.dtypes
    temp_dict['# значений'] = data.describe(include='all').loc['count'].astype(int)
    temp_dict['# пропусков(NaN)'] = data.isnull().sum().values 
    temp_dict['# уникальных'] = data.nunique().values
    
    if not short:
        temp_dict['в первой строке'] =data.loc[0].values
        temp_dict['во второй строке'] = data.loc[1].values
        temp_dict['в третьей строке'] = data.loc[2].values
        
    if full :
        temp_dict['минимум'] = data.describe(include='all').loc['min']
        temp_dict['среднее'] = data.describe(include='all').loc['mean']
        temp_dict['макс'] = data.describe(include='all').loc['max']
        temp_dict['медиана'] = data.describe(include='all').loc['50%']
        
    temp_df = pd.DataFrame.from_dict(temp_dict, orient='index')
    display(temp_df.T)

    return

In [ ]:
def column_info(data: pd.DataFrame, column: str) -> None:
    '''Функция для вывода информации для номинативных признаков.
    data - набор исходных данных
    column - наименование признака
    
    Функция производит расчет ключевых значений номинативного признака и выводит их в stdout.
    
    '''
    
    n = 10 # Параметр для оценки наиболее часто встречающихся значений
    print(f'Проведем анализ для переменной {column}')
    
    print('Тип данных: ', data[column].dtype)
    print('Всего значений:', data.shape[0])
    print('Пустых значений:', data.shape[0] - data[column].notnull().sum())
    print(f'Значений, упомянутых более {n} раз:', (data[column].value_counts() > n).sum())
    print('Уникальных значений:', data[column].nunique())
      
    display(pd.DataFrame(data[column].value_counts()).head(10))
    

def plot_column_info(data: pd.DataFrame, column: str) -> None:  
    '''Функция для вывода информации для колличественных переменных.
    data - набор исходных данных
    column - наименование признака
    
    Функция производит расчет ключевых значений колличественного признака.
    Определяет наличие выбросов. Выводит полученные данные в stdout. 
    Риует гистограмму исходных значений и значениий ограниченных границами выбросов IQR.
    
    '''
    
    max_value = data[column].max()
    min_value = data[column].min()
    perc25 = data[column].quantile(0.25)
    perc75 = data[column].quantile(0.75)
    IQR = perc75 - perc25
    lower_limt = perc25 - 1.5*IQR
    upper_limit = perc75 + 1.5*IQR

    if max_value <= upper_limit and min_value >= lower_limt:
        outliers_status = 'Выбросов Нет.'
    else:
        outliers_status = 'Есть выбросы.'

    print('Всего значений:', data.shape[0])
    print('Пустых значений:', data.shape[0] - data[column].notnull().sum())
    
    print('25-й перцентиль: {}, 75-й перцентиль: {}, IQR: {},'.format(perc25, perc75, IQR))
    print(f'Границы значений: [{min_value}, {max_value}],')
    print('Границы выбросов: [{f}, {l}].'.format(f=lower_limt, l=upper_limit))
    print(outliers_status)
    
    plt.rcParams['figure.figsize'] = (10,7)
    
    data[column].loc[data[column].between(lower_limt, upper_limit)].hist(bins=20,
                                                                         range=(min_value, max_value), 
                                                                         label='IQR')

    data[column].hist(alpha=0.5, 
                      bins=20, 
                      range=(min_value, max_value), 
                      label='Исходные значения')

    plt.legend();

In [ ]:
def plot_column_info_log(data: pd.DataFrame, column: str) -> None:  
    '''Функция для вывода информации для колличественных переменных.
    data - набор исходных данных
    column - наименование признака
    
    Функция производит расчет ключевых значений колличественного признака.
    Определяет наличие выбросов. Выводит полученные данные в stdout. 
    Риует гистограмму исходных значений, boxplot исходных значений, гистограмму логарифма исходных значений 
    и boxplot для них.
    
    '''
    
    max_value = data[column].max()
    min_value = data[column].min()
    perc25 = data[column].quantile(0.25)
    perc75 = data[column].quantile(0.75)
    IQR = perc75 - perc25
    lower_limt = perc25 - 1.5*IQR
    upper_limit = perc75 + 1.5*IQR
    
    if max_value <= upper_limit and min_value >= lower_limt:
        outliers_status = 'Выбросов Нет.'
    else:
        outliers_status = 'Есть выбросы.'
    
    outliers_num = data.query(f'{column} < {lower_limt} or {column} > {upper_limit}').shape[0]
    
    print('Всего значений:', data.shape[0])
    print('Пустых значений:', data.shape[0] - data[column].notnull().sum())
    
    print('25-й перцентиль: {}, 75-й перцентиль: {}, IQR: {},'.format(perc25, perc75, IQR))
    print(f'Границы значений: [{min_value}, {max_value}],')
    print('Границы выбросов: [{f}, {l}].'.format(f=lower_limt, l=upper_limit))
    print(outliers_status)
    print('Количество выбросов:', outliers_num)
    
    plt.rcParams['figure.figsize'] = (12,4)
    
    ser_iqr = data[column].loc[data[column].between(lower_limt, upper_limit)]
    ser_act = data[column]
    ser_log = data[column].apply(lambda x: math.log(x + 1))
    
    fig = plt.figure()
    st = fig.suptitle(f'Гистограммы и box-plot для признака \'{column}\' и log(\'{column}\')', fontsize='x-large')
    
    ax_1 = fig.add_subplot(1, 4, 1)
    ax_2 = fig.add_subplot(1, 4, 2)
    ax_3 = fig.add_subplot(1, 4, 3)
    ax_4 = fig.add_subplot(1, 4, 4)
    
    ax_1.hist(ser_act, bins=15)
    ax_1.set_title(f'\'{column}\'', loc = 'right', fontsize=10)
    ax_1.spines['top'].set_visible(False)
    ax_1.spines['right'].set_visible(False)
    
    ax_2.boxplot(ser_act)
    ax_2.set_title(f'boxplot of \'{column}\'', loc = 'left', fontsize=10)
    ax_2.spines['top'].set_visible(False)
    ax_2.spines['right'].set_visible(False)
    ax_2.spines['bottom'].set_visible(False)
    ax_2.spines['left'].set_visible(False)
    
    ax_3.hist(ser_log, bins=15)
    ax_3.set_title(f'log(\'{column}\')', loc = 'right', fontsize=10)
    ax_3.spines['top'].set_visible(False)
    ax_3.spines['right'].set_visible(False)
    
    ax_4.boxplot(ser_log)
    ax_4.set_title(f'boxplot of log(\'{column}\')', loc = 'left', fontsize=10)
    ax_4.spines['top'].set_visible(False)
    ax_4.spines['right'].set_visible(False)
    ax_4.spines['bottom'].set_visible(False)
    ax_4.spines['left'].set_visible(False)
    
    
    # shift subplots down:
    st.set_y(0.99)
    fig.subplots_adjust(top=0.85)
    
    plt.legend()
    plt.show();

In [ ]:
def replace_nan(data_in: pd.DataFrame, column: str, typ: str, new_value='') -> pd.DataFrame:
    '''Функция возвращает датасет, в котором произведена замена пустых значений признака и добавлен новый
    признак, в котором сохраняется информаця о том, в какой строке было пустое значение
    data_in - набор исходных данных
    column - наименование признака
    typ - тип замены, может принимать значения: 'mode', 'median', 'mean', 'new_value'
    
    Создается новый признак с наименованием (column)_isnan, в котором сохраняется информаця 
    о том, в какой строке пустое значение признака (column)
    В зависимости от типа замены (typ) функция определяет значение, 
    на которое будет производится замена.
    Все значения признака (column) в наборе данных (data_in) заменяются на вычисленное значение.
    Функция возращает откорректированный набор данных, изменений в исходном наборе нет.
    
    '''
    
    data = data_in.copy()
    
    print(f'В новый признак \'{column}_isnan\' охраняем информацию, в каком наблюдении указано NaN')
    data.loc[:, column + '_isnan'] = pd.isna(data[column]).astype('uint8')
    
    if typ == 'mode':
        m = data.loc[:, column].mode()[0]
        s = 'самым распространенным значением '
    elif typ == 'median':
        m = data.loc[:, column].median()
        s = 'медианой'
    elif typ == 'mean':
        m = data.loc[:, column].mean()
        s = 'средневзвешенным значением'
    elif typ == 'new_value':
        m = new_value
        s = 'указанным значением'
    else:
        return None
    
    print(f'Заполним пустые значения признака {column} {s} {m}')
    data.loc[:, column] = data[column].fillna(m)
    
    return data

In [ ]:
def all_metrics(d_y_true, d_y_pred, d_y_pred_prob):
        
    temp_dict = {}
    temp1 = accuracy_score(d_y_true, d_y_pred)
    temp_dict['accuracy'] = [temp1, '(TP+TN)/(P+N)']

    temp1 = balanced_accuracy_score(d_y_true, d_y_pred)
    temp_dict['balanced accuracy'] = [temp1, 'сбалансированная accuracy']
    
    temp1 = precision_score(d_y_true, d_y_pred)
    temp_dict['precision'] = [temp1, 'точность = TP/(TP+FP)']
    
    temp1 = recall_score(d_y_true, d_y_pred)
    temp_dict['recall'] = [temp1, 'полнота = TP/P']
    
    temp1 = f1_score(d_y_true, d_y_pred)
    temp_dict['f1_score'] = [temp1, 'среднее гармоническое точности и полноты']
    
    temp1 = roc_auc_score(d_y_true, d_y_pred_prob)
    temp_dict['roc_auc'] = [temp1, 'Area Under Curve - Receiver Operating Characteristic']    
    
    temp_df = pd.DataFrame.from_dict(temp_dict, orient='index', columns=['Значение', 'Описание'])
    display(temp_df)

    return


def model_coef(d_columns, d_model_coef_0):

    temp_dict = {}
    temp_dict['имя признака'] = d_columns
    temp_dict['коэффициент модели'] = d_model_coef_0
    temp_dict['модуль коэф'] = abs(temp_dict['коэффициент модели'])
    temp_df = pd.DataFrame.from_dict(temp_dict, orient='columns')
    temp_df = temp_df.sort_values(by='модуль коэф', ascending=False)
    temp_df.reset_index(drop=True,inplace=True)
    
    return temp_df.loc[:,['имя признака','коэффициент модели']]

In [ ]:
def plot_cv_metrics(cv_metrics):
    avg_f1_train, std_f1_train = cv_metrics['train_score'].mean(), cv_metrics['train_score'].std()
    avg_f1_valid, std_f1_valid = cv_metrics['test_score'].mean(), cv_metrics['test_score'].std()
    print('[train] F1-score = {:.2f} +/- {:.2f}'.format(avg_f1_train, std_f1_train))
    print('[valid] F1-score = {:.2f} +/- {:.2f}'.format(avg_f1_valid, std_f1_valid))
    
    plt.figure(figsize=(15, 5))

    plt.plot(cv_metrics['train_score'], label='train', marker='.')
    plt.plot(cv_metrics['test_score'], label='valid', marker='.')

    plt.ylim([0., 1.]);
    plt.xlabel('CV iteration', fontsize=15)
    plt.ylabel('F1-score', fontsize=15)
    plt.legend(fontsize=15)
    
    
def vis_cross_val_score(d_name_metric, d_vec, d_value_metric, d_my_font_scale):
    num_folds = len(d_vec['train_score'])
    avg_metric_train, std_metric_train = d_vec['train_score'].mean(), d_vec['train_score'].std()
    avg_metric_test, std_metric_test = d_vec['test_score'].mean(), d_vec['test_score'].std()

    plt.style.use('seaborn-paper')
    sns.set(font_scale=d_my_font_scale)
    color_text = plt.get_cmap('PuBu')(0.85)

    plt.figure(figsize=(12, 6))
    plt.plot(d_vec['train_score'], label='тренировочные значения', marker='.', color= 'darkblue')
    plt.plot([0,num_folds-1], [avg_metric_train, avg_metric_train], color='blue', label='среднее трен. значений ', marker='.', lw=2, ls = '--')

    plt.plot(d_vec['test_score'], label='тестовые значения', marker='.', color= 'red')
    plt.plot([0,num_folds-1], [avg_metric_test, avg_metric_test], color='lightcoral', label='среднее тест. значений ', marker='.', lw=2, ls = '--')

    plt.plot([0,num_folds-1], [d_value_metric, d_value_metric], color='grey', label='значение метрики до CV', marker='.', lw=3)

    # plt.xlim([1, num_folds])
    y_max = max(avg_metric_train,avg_metric_test) + 1.5*max(std_metric_train,std_metric_test)
    y_min = min(avg_metric_train,avg_metric_test) - 3*max(std_metric_train,std_metric_test)
    plt.ylim([y_min, y_max])
    plt.xlabel('номер фолда', fontsize=15, color = color_text)
    plt.ylabel(d_name_metric, fontsize=15, color = color_text)
    plt.title(f'Кросс-валидация по метрике {d_name_metric} на {num_folds} фолдах', color = color_text, fontsize=17)
    plt.legend(loc="lower right", fontsize=11)
    y_min_text = y_min +0.5*max(std_metric_train,std_metric_test)
    plt.text(0, y_min_text, f'{d_name_metric} на трейне = {round(avg_metric_train,3)} +/- {round(std_metric_train,3)} \n{d_name_metric} на тесте    = {round(avg_metric_test,3)} +/- {round(std_metric_test,3)} \n{d_name_metric} до CV        = {round(d_value_metric,3)}', fontsize = 15)
    plt.show()
    return

In [ ]:
# Класс-помощник для красивого отображения данных о модели. Взял у https://www.kaggle.com/ekalachev
class ModelInspector():
    def __init__(self, model, X, y):
        self.model = model
        self.X = X
        self.y = y

    def _plot_confusion_matrix(self, y_pred, ax):
        tn, fp, fn, tp = confusion_matrix(self.y, y_pred).ravel()

        matrix = np.eye(2)
        matrix_annot = [[f'TP\n{tp}', f'FP\n{fp}'], [f'FN\n{fn}', f'TN\n{tn}']]

        sns.heatmap(matrix, annot=matrix_annot, annot_kws={"size": 20}, fmt='', cmap='Greens', cbar=False,
                    xticklabels=['', 'Good client'], yticklabels=['Bad client', ''], ax=ax)

    def _plot_metrics(self, y_pred, roc_auc, ax):
        matrix = np.array([[1, 0, 1, 0, 1]]).T

        matrix_annot = np.array([[
            f'ROC AUC: {roc_auc:.4f}',
            f'Balanced accuracy: {balanced_accuracy_score(self.y, y_pred):.3f}',
            f'F1-score: {f1_score(self.y, y_pred):.3f}',
            f'Precision score: {precision_score(self.y, y_pred):.3f}',
            f'Recall score: {recall_score(self.y, y_pred):.3f}'
        ]]).T

        sns.heatmap(matrix, annot=matrix_annot, fmt='', cbar=False, yticklabels=[],
                    xticklabels=[], annot_kws={'size': 16, 'ha': 'center'}, cmap='GnBu', ax=ax)

    def _plot_logistic_regression(self, probs, ax):
        fpr, tpr, threshold = roc_curve(self.y, probs)

        ax.plot([0, 1], label='Baseline', linestyle='--')
        ax.plot(fpr, tpr, label='Regression')
        ax.set_ylabel('True Positive Rate')
        ax.set_xlabel('False Positive Rate')
        ax.legend(loc='lower right')

    def inspect(self, size=5):
        y_pred = self.model.predict(self.X)
        probs = self.model.predict_proba(self.X)[:, 1]
        roc_auc = roc_auc_score(self.y, probs)

        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(3*size, size))

        fig.suptitle(f'Model inspection. ROC AUC: {roc_auc:.4f}', fontsize=20)

        ax1.set_title('Logistic Regression')
        ax2.set_title('Confusion matrix')
        ax3.set_title('Metrics')

        # Plot logistic regression
        self._plot_logistic_regression(probs, ax1)

        # Plot confusion matrix
        self._plot_confusion_matrix(y_pred, ax2)

        # plot metrics
        self._plot_metrics(y_pred, roc_auc, ax3)

        plt.show()

In [ ]:
def get_stat_dif(data: pd.DataFrame, column: str, target: str) -> bool:
    '''Функция определяет, есть ли статистически значимые различия для признака.
    data - набор исходных данных
    column - наименование признака
    target - наименование целевого признака
    
    Функция проверяет, есть ли статистическая разница в распределении целевого признака (target)
    по номинативному признаку (column) в наборе данных (data). 
    Проверка осуществляется с помощью теста Стьюдента. 
    Проверяется нулевая гипотеза о том, что распределения целевого признака (target)
    по различным парам значений номинативного признака (column) неразличимы.
    Пары определяются из различных комбинаций n самых часто встречающихся занчений признака (column).
    Если различия найдены, то информация об этом выводится в stdout.
    
    '''
    
    n = 10 # Параметр для оценки наиболее часто встречающихся значений
    cols = data.loc[:, column].value_counts().index[:n]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(data.loc[data.loc[:, column] == comb[0], target], 
                     data.loc[data.loc[:, column] == comb[1], target]).pvalue \
            <= 0.05/len(combinations_all): # Учли поправку Бонферони
            print('Найдены статистически значимые различия для признака', column)
            return True
    
    return False

## Проведем предварительный анализ данных

In [ ]:
# Выведем общую информацию по датасету
data_full_info(data, short=True)

In [ ]:
# Заведем 4 списка под разные  типы признаков
# Временные признаки 
time_cols = []

# Бинарные признаки 
bin_cols = []

# Категориальные признаки
cat_cols = []

# Колличественные признаки 
num_cols = []

# Лишние признаки
del_cols = []

# целевые признаки
target_cols = []

Признак **body_type**

In [ ]:
column_info(data, 'body_type')

In [ ]:
# Посмотрим на объек с пустым полем в body_type
data.query('body_type != body_type')

In [ ]:
# Выносим в блок предобработки

# Данный объект имеет очень много пропусков, в том числе и в полях model_date, name, number_of_doors, engine_power, drive_type, vehicle_transmission
# Удалим его
data.dropna(subset=['body_type'], inplace=True)

In [ ]:
# Еще раз посмотрим общую информацию по датасету
data_full_info(data, short=True)

In [ ]:
# Выносим в блок предобработки

# Приведем значения признака body_type к нижнему регистру
data.loc[:, 'body_type'] = data['body_type'].apply(lambda x: str(x).lower())

# отнесем признак к категориальным
cat_cols.append('body_type')

In [ ]:
column_info(data, 'body_type')

Пока оставим данный признак так. 

**Идея на будущее:** можно оставить только внедорожник, седан, лифтбек и т.д. без доп.информации

Признак **'brand'**

In [ ]:
column_info(data, 'brand')

In [ ]:
# отнесем признак к категориальным
cat_cols.append('brand')

Все хорошо. Оставляем признак как есть.

Признак **'color'**

In [ ]:
column_info(data, 'color')

In [ ]:
# отнесем признак к категориальным
cat_cols.append('color')

Все хорошо. Оставляем признак как есть.

Признак **'pts'**

In [ ]:
column_info(data, 'pts')

In [ ]:
# Выносим в блок предобработки

# Есть пустые значения. Заполняем пустые значения самым распространенным значением
data = replace_nan(data, 'pts', 'mode')

In [ ]:
# отнесем признак к бинарным
bin_cols.append('pts_isnan')

In [ ]:
column_info(data, 'pts')

In [ ]:
# Преобразуем education в целочисленный формат. 
pts_dict = {'ORIGINAL': 1, 'DUPLICATE': 0}
data.loc[:, 'pts'] = data['pts'].map(pts_dict)

In [ ]:
# отнесем признак к бинарным
bin_cols.append('pts')

Все хорошо. Оставляем признак как есть.

Признак **'fuel_type'**

In [ ]:
column_info(data, 'fuel_type')

In [ ]:
# отнесем признак к категориальным
cat_cols.append('fuel_type')

Все хорошо. Оставляем признак как есть.

Признак **'mileage'**

In [ ]:
plot_column_info_log(data, 'mileage')

In [ ]:
# Выносим в блок предобработки

# Возьмем логарифм от признака
data.loc[:, 'mileage'] = np.log(data['mileage'] + 1)

# Добавим новый признак 'is_new' Новая машина или б/у
data.loc[:, 'is_new'] = data['mileage'].apply(lambda x: 0 if x > 0 else 1)

С выбросами пока ничего не делаем

In [ ]:
# отнесем признак к количественным
num_cols.append('mileage')

# отнесем признак к бинарным
bin_cols.append('is_new')

Признак **'model_date'**

In [ ]:
plot_column_info(data, 'model_date')

С выбросами пока ничего не делаем. Автомобили с таким ранним годом выпуска возможны.

In [ ]:
# отнесем признак к количественным, можно было бы и к временным, но пока не вижу смысла
num_cols.append('model_date')

Все хорошо. Оставляем признак как есть.

Признак **'name'**

Данный признаак составной. Основная информация содержится в признаках 'engine_displacement', 'vehicle_transmission', 'engine_power', 'drive_type'.
В некоторых случаях еще дополнительно указывают модель двигателся и модель коробки передач

In [ ]:
# посмотрим на признак
data[['name', 'engine_displacement', 'vehicle_transmission', 'engine_power', 'drive_type']].sample(10)

Уберем признак.

**Идея на будущее:** можно добавить два признака - модель двигателся и модель коробки передач

In [ ]:
# Уберем признак.
del_cols.append('name')

Признак **'number_of_doors'**

In [ ]:
plot_column_info_log(data, 'number_of_doors')

In [ ]:
# отнесем признак к количественным
num_cols.append('number_of_doors')

Все хорошо. Оставляем признак как есть.

Признак **'engine_power'**

In [ ]:
column_info(data, 'engine_power')

In [ ]:
# Выносим в блок предобработки

# приведем к формату int
data.loc[:, 'engine_power'] = data['engine_power'].astype('int')

In [ ]:
plot_column_info_log(data, 'engine_power')

In [ ]:
# Выносим в блок предобработки

# Возьмем логарифм от признака
data.loc[:, 'engine_power'] = np.log(data['engine_power'] + 1)

In [ ]:
# отнесем признак к количественным
num_cols.append('engine_power')

Все хорошо. С выбросами ничего не делаем, так как машины с мощностью двигателя выше 412 л.с. и до 800 л.с. вполне могут быть.

Признак **'wheel'**

In [ ]:
column_info(data, 'wheel')

In [ ]:
# Преобразуем wheel в целочисленный формат. 
wheel_dict = {'LEFT': 1, 'RIGHT': 0}
data.loc[:, 'wheel'] = data['wheel'].map(wheel_dict)

In [ ]:
# отнесем признак к бинарным
bin_cols.append('wheel')

С признаком все хорошо. Оставляем как есть.

Признак **'drive_type'**

In [ ]:
column_info(data, 'drive_type')

In [ ]:
# отнесем признак к категориальным
cat_cols.append('drive_type')

С признаком все хорошо. Оставляем как есть.

Признак **'owners'**

In [ ]:
plot_column_info_log(data, 'owners')

In [ ]:
# В поле Владельцы есть много пропусков. Но по большей части пропуски указаны для машин с нулевым пробегом
data.query('owners != owners')['mileage'].value_counts()

In [ ]:
# Вынесем в блок предобработки

# Заполним пропуски в поле owners 0, если проьег нулевой.
data.loc[data.query('owners != owners and mileage == 0').index, 'owners'] = 0

# Заполним пропуски в поле owners медианным значением
data = replace_nan(data, 'owners', 'median')

In [ ]:
# отнесем признак к количественным
num_cols.append('owners')

# новый признак owners_isnan отнесем к бинарным
bin_cols.append('owners_isnan')

С признаком все хорошо. Оставляем как есть.

Признак **'vehicle_transmission'**

In [ ]:
column_info(data, 'vehicle_transmission')

In [ ]:
# отнесем признак к категориальным
cat_cols.append('vehicle_transmission')

С признаком все хорошо. Оставляем как есть.

Признак **'production_date'**

In [ ]:
plot_column_info(data, 'production_date')

In [ ]:
# отнесем признак к количественным. можно было бы и к временным, но пока не вижу смысла
num_cols.append('production_date')

С признаком все хорошо. Оставляем как есть.

Признак **'vendor'**

In [ ]:
column_info(data, 'vendor')

In [ ]:
# Посмотрим на выделяющуюся машину из Америки
model_name = data.query('vendor == "AMERICAN"')['model_name'].values[0]
data.query(f'model_name == "{model_name}"')

По всей видимости произошла ошибка при регистрации на сайте. Поправим значение признака на 'EUROPEAN'

In [ ]:
# Вынесем в блок предобработки

# Поправим значение признака на 'EUROPEAN'
index = data.query('vendor == "AMERICAN"').index
data.loc[index, 'vendor'] = 'EUROPEAN'

In [ ]:
column_info(data, 'vendor')

In [ ]:
# Преобразуем vendor в целочисленный формат. 
vendor_dict = {'EUROPEAN': 1, 'JAPANESE': 0}
data.loc[:, 'vendor'] = data['vendor'].map(vendor_dict)

In [ ]:
# отнесем признак к бинарным
bin_cols.append('vendor')

С признаком все хорошо. Оставляем как есть.

Признак **'model_name'**

In [ ]:
column_info(data, 'model_name')

In [ ]:
# отнесем признак к категориальным
cat_cols.append('model_name')

С признаком все хорошо. Оставляем как есть.

Признак **'engine_displacement'**

In [ ]:
plot_column_info_log(data, 'engine_displacement')

In [ ]:
# Выносим в блок предобработки

# Возьмем логарифм от признака
data.loc[:, 'engine_displacement'] = np.log(data['engine_displacement'] + 1)

In [ ]:
# отнесем признак к количественным.
num_cols.append('engine_displacement')

Оставляем как есть. С выбросами ничего не делаем, 0 объем указан для электромобилей, что логично. Также есть машины с большим объемом двигателя (более 5 л.), это вполне реально, особенно для спорткаров или больших внедорожников.

Признак **'ownership'**

In [ ]:
column_info(data, 'ownership')

In [ ]:
# Временно не будем рассматривать признаки 'ownership', 'complectation_dict', 'equipment_dict'

del_cols.append('ownership')
del_cols.append('complectation_dict')
del_cols.append('equipment_dict')

Рассмотрим признак **'parsing_unixtime'**

In [ ]:
column_info(data, 'parsing_unixtime')

In [ ]:
# Пустых значений нет. Переведем в формат времени
data.loc[:, 'parsing_unixtime'] = data['parsing_unixtime'].apply(lambda x: datetime.fromtimestamp(x))

In [ ]:
# определим к временным признакам
time_cols.append('parsing_unixtime')

In [ ]:
# Добавим новый признак 'usd_rub_rate_date' - курс доллара к рублю на момент выгрузки данных
usd_rub_rate_date = {
    '10.07.2021': 74.4675, '26.10.2020': 76.4667, '25.10.2020': 76.4667,
    '24.10.2020': 76.4667, '21.10.2020': 77.7780, '20.10.2020': 77.9241, '19.10.2020': 77.9644
}
data.loc[:, 'usd_rub_rate_date'] = data['parsing_unixtime'].dt.strftime('%d.%m.%Y').map(usd_rub_rate_date)

In [ ]:
# отнесем признак к техническим, пометим как к удалению.
del_cols.append('usd_rub_rate_date')

Рассмотрим целевой признак **'price'**

In [ ]:
plot_column_info_log(data.query('sample == 1'), 'price')

In [ ]:
# Выносим в блок предобработки

# Заполним пустые значения медианным значение для данной марки
index_nan_price = data.query('sample == 1 and price != price').index
median_price_model = data.query('sample == 1 and price == price').groupby('model_name')['price'].median()
data.loc[index_nan_price, 'price'] = data.loc[index_nan_price, 'model_name'].map(median_price_model)

In [ ]:
plot_column_info_log(data.query('sample == 1'), 'price')

In [ ]:
# Выносим в блок предобработки

# Добавим новый признак - целевой признак в usd
data.loc[:, 'price_usd'] = data['price'] / data['usd_rub_rate_date']

In [ ]:
plot_column_info_log(data.query('sample == 1'), 'price_usd')

In [ ]:
# Выносим в блок предобработки

# Возьмем логарифм от целевого признака
data.loc[:, 'price_log'] = np.log(data['price'] + 1)
data.loc[:, 'price_usd_log'] = np.log(data['price_usd'] + 1)

# определим все 4 признака к целевым
target_cols += ['price', 'price_usd', 'price_log', 'price_usd_log']

In [ ]:
display(data.columns)
len(data.columns)

In [ ]:
display(time_cols)
display(bin_cols)
display(cat_cols)
display(num_cols)
display(del_cols)
len(time_cols + bin_cols + cat_cols + num_cols + del_cols + ['price', 'price_usd', 'sample'])

In [ ]:
# Выведем информацию после изменений
data_full_info(data[cat_cols + bin_cols], full=False, short=False)

In [ ]:
# Выведем информацию после изменений
data_full_info(data[num_cols], full=False, short=False)

In [ ]:
def clear_and_prepare_data(data_in: pd.DataFrame) -> pd.DataFrame:
    # Создадим в блок предобработки
    
    data = data_in.copy()
    
    # Данный объект имеет очень много пропусков, в том числе и в полях model_date, name, number_of_doors, engine_power, drive_type, vehicle_transmission
    # Удалим его
    data = data.dropna(subset=['body_type'])
    # Приведем значения признака body_type к нижнему регистру
    data.loc[:, 'body_type'] = data['body_type'].apply(lambda x: str(x).lower())

    # Есть пустые значения. Заполняем пустые значения самым распространенным значением
    data = replace_nan(data, 'pts', 'mode')
    # Преобразуем education в целочисленный формат. 
    pts_dict = {'ORIGINAL': 1, 'DUPLICATE': 0}
    data.loc[:, 'pts'] = data['pts'].map(pts_dict)

    # Возьмем логарифм от признака
    data.loc[:, 'mileage'] = np.log(data['mileage'] + 1)
    # Добавим новый признак 'is_new' Новая машина или б/у
    data.loc[:, 'is_new'] = data['mileage'].apply(lambda x: 0 if x > 0 else 1)

    # приведем к формату int
    data.loc[:, 'engine_power'] = data['engine_power'].astype('int')
    # Возьмем логарифм от признака
    data.loc[:, 'engine_power'] = np.log(data['engine_power'] + 1)

    # Преобразуем wheel в целочисленный формат. 
    wheel_dict = {'LEFT': 1, 'RIGHT': 0}
    data.loc[:, 'wheel'] = data['wheel'].map(wheel_dict)

    # Заполним пропуски в поле owners 0, если проьег нулевой.
    data.loc[data.query('owners != owners and mileage == 0').index, 'owners'] = 0
    # Заполним пропуски в поле owners медианным значением
    data = replace_nan(data, 'owners', 'median')

    # Поправим значение признака на 'EUROPEAN'
    index = data.query('vendor == "AMERICAN"').index
    data.loc[index, 'vendor'] = 'EUROPEAN'
    # Преобразуем vendor в целочисленный формат. 
    vendor_dict = {'EUROPEAN': 1, 'JAPANESE': 0}
    data.loc[:, 'vendor'] = data['vendor'].map(vendor_dict)

    # Возьмем логарифм от признака
    data.loc[:, 'engine_displacement'] = np.log(data['engine_displacement'] + 1)

    # Пустых значений нет. Переведем в формат времени
    data.loc[:, 'parsing_unixtime'] = data['parsing_unixtime'].apply(lambda x: datetime.fromtimestamp(x))

    # Добавим новый признак 'usd_rub_rate_date' - курс доллара к рублю на момент выгрузки данных
    usd_rub_rate_date = {
        '10.07.2021': 74.4675, '26.10.2020': 76.4667, '25.10.2020': 76.4667,
        '24.10.2020': 76.4667, '21.10.2020': 77.7780, '20.10.2020': 77.9241, '19.10.2020': 77.9644
    }
    data.loc[:, 'usd_rub_rate_date'] = data['parsing_unixtime'].dt.strftime('%d.%m.%Y').map(usd_rub_rate_date)

    # Заполним пустые значения медианным значение для данной марки
    index_nan_price = data.query('sample == 1 and price != price').index
    median_price_model = data.query('sample == 1 and price == price').groupby('model_name')['price'].median()
    data.loc[index_nan_price, 'price'] = data.loc[index_nan_price, 'model_name'].map(median_price_model)

    # Добавим пцелевой ризнак в валюте usd
    data.loc[:, 'price_usd'] = data['price'] / data['usd_rub_rate_date']
    # Возьмем логарифм от целевого признака
    data.loc[:, 'price_log'] = np.log(data['price'] + 1)
    data.loc[:, 'price_usd_log'] = np.log(data['price_usd'] + 1)
    
    return data

In [ ]:
# Загрузим данные
train, test, sample_submission = load_data_frames()

# Согласуем наименования признаков
rename_columns_train(train)
rename_columns_test(test)

# Согласуем форматы признаков
prepare_columns_format_train(train)
prepare_columns_format_test(test)

# Объединим датасеты в один
data = concatenate_data_frames(train, test)

# Почистим и подготовим датасет
data = clear_and_prepare_data(data)

In [ ]:
# Выведем информацию после изменений
data_full_info(data[cat_cols + bin_cols], full=False, short=False)

In [ ]:
# Выведем информацию после изменений
data_full_info(data[num_cols], full=False, short=False)

## Оценка корреляций

In [ ]:
# Оценим корреляцию Пирсона для непрерывных переменных 
plt.figure(figsize = (10,7))
sns.heatmap(data[num_cols + ['price','price_log']].corr().abs(), vmin=0, vmax=1, annot=True)

In [ ]:
num_cols.remove('model_date')

## Посмотрим на значимость колличественных переменных
Для оценки значимости числовых переменных будем использовать функцию f_classif из библиотеки sklearn.

В качестве меры значимости мы будем использовать значение f-статистики. Чем значение статистики выше, тем меньше вероятность того, что средние значения не отличаются, и тем важнее данный признак для нашей линейной модели.

In [ ]:
df = data.query('sample == 1')
imp_num = pd.Series(f_classif(df[num_cols], df['price_log'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh', title='Значимость непрерывных переменных по ANOVA F test')

## Посмотрим на значимость категориальных и бинарных переменных

Для оценки значимости категориальных и бинарных переменных будем использовать функцию mutual_info_classif из библиотеки sklearn. Данная функция опирается на непараметрические методы, основанные на оценке энтропии в группах категориальных переменных.

In [ ]:
nom_cols = cat_cols + bin_cols
delete_cols = []

for col in nom_cols:
    if not get_stat_dif(data.query('sample == 1'), col, 'price_log'):
        delete_cols.append(col)
        
print('Признаки на удаление: ', delete_cols)

# 5. Data Preprocessing

Перед обучением регрессии необходимо сделать две вещи:

 - Стандартизировать числовые признаки
 - Применить Label Encoding подход к категориальным признакам
 - Сделать разбиение на тестовую и валидационную выборки

## Label Encoding для кактегориальных признаков

In [ ]:
# применим подход Label Encoding для категориальных признаков
for colum in cat_cols:
    data[colum] = data[colum].astype('category').cat.codes

In [ ]:
data.sample(4)

## Стандартизируем числовые переменные:

In [ ]:
# Стандартизация числовых переменных
scaler = StandardScaler()

index = data.query('sample == 1')[num_cols].index
values_income = data.query('sample == 1')[num_cols].values
values_norm = scaler.fit_transform(values_income)
data.loc[index, num_cols] = values_norm

index = data.query('sample == 0')[num_cols].index
values_income = data.query('sample == 0')[num_cols].values
values_norm = scaler.transform(values_income)
data.loc[index, num_cols] = values_norm

## Подготовка выборок

In [ ]:
X = data.query('sample == 1').drop(columns=time_cols + del_cols + ['price','price_usd','usd_rub_rate_date'])
X_sub = data.query('sample == 0').drop(columns=time_cols + del_cols + ['price','price_usd','usd_rub_rate_date'])

y = data.query('sample == 1')['price_usd'].values

In [ ]:
def get_columns():
    time_cols = ['parsing_unixtime']
    bin_cols = ['pts_isnan', 'pts', 'is_new', 'wheel', 'owners_isnan', 'vendor']
    cat_cols = [
        'body_type', 'brand', 'color', 'fuel_type',
        'drive_type', 'vehicle_transmission', 'model_name'
    ]
    num_cols = [
        'mileage', 'model_date', 'number_of_doors', 'engine_power',
        'owners', 'production_date', 'engine_displacement']
    del_cols = [
        'name', 'ownership', 'complectation_dict',
        'equipment_dict', 'usd_rub_rate_date'
    ]
    target_cols = ['price','price_usd','price_log','price_usd_log']
    
    return time_cols, bin_cols, cat_cols, num_cols, del_cols, target_cols

In [ ]:
def prepare_data_before_split(data_in: pd.DataFrame, column_target: str) -> pd.DataFrame:
    # Объединим подготовку в один модуль
    data = data_in.copy()
    time_cols, bin_cols, cat_cols, num_cols, del_cols, target_cols = get_columns()
    
    # применим подход Label Encoding для категориальных признаков
    for colum in cat_cols:
        data[colum] = data[colum].astype('category').cat.codes
        
    # Стандартизация числовых переменных
    scaler = StandardScaler()

    index = data.query('sample == 1')[num_cols].index
    values_income = data.query('sample == 1')[num_cols].values
    values_norm = scaler.fit_transform(values_income)
    data.loc[index, num_cols] = values_norm

    index = data.query('sample == 0')[num_cols].index
    values_income = data.query('sample == 0')[num_cols].values
    values_norm = scaler.transform(values_income)
    data.loc[index, num_cols] = values_norm
    
    #
    X = data.query('sample == 1').drop(columns=time_cols + del_cols + target_cols + ['sample'])
    X_sub = data.query('sample == 0').drop(columns=time_cols + del_cols + target_cols + ['sample'])
    y = data.query('sample == 1')[column_target]
    
    return X, X_sub, y

In [ ]:
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# 6. Model

Загрузим и подготовим все данные снуля

In [ ]:
# Загрузим данные
train, test, sample_submission = load_data_frames()

# Согласуем наименования признаков
rename_columns_train(train)
rename_columns_test(test)

# Согласуем форматы признаков
prepare_columns_format_train(train)
prepare_columns_format_test(test)

# Объединим датасеты в один
data = concatenate_data_frames(train, test)

# Почистим и подготовим датасет
data = clear_and_prepare_data(data)

# сделаем предобработку
X, X_sub, y = prepare_data_before_split(data, 'price_log')

# Разобъем выборку на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

## CatBoost

У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

In [ ]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_2_baseline.model')

In [ ]:
# Получим предсказанные значения для валидации модели
y_true = y_test
predict_test = model.predict(X_test)

print(f"Точность модели по метрике MAPE: {(mape(y_true, predict_test))*100:0.2f}%")
print(f"Точность модели по метрике MAPE если перевести в валюту: {(mape(np.exp(y_true) - 1, np.exp(predict_test) - 1))*100:0.2f}%")

In [ ]:
# Получим предсказанные значения submission
predict_submission = np.exp(model.predict(X_sub)) - 1

# В качестве целевой переменной мы использовали цену в долларах. Надо перевести предсказанные значения submission обратно в рубли
#usd_rub_rate_submission = data.query('sample == 0')['usd_rub_rate_date'].values
#predict_submission = np.round(predict_submission * usd_rub_rate_submission, 2)


In [ ]:
from sklearn.metrics import fbeta_score, make_scorer

In [ ]:
score = make_scorer(mape, greater_is_better=False)

cv_metrics = cross_validate(model, X_test, y_test, cv=5, scoring=score, return_train_score=True)

vis_cross_val_score('MAPE', cv_metrics, -0.77, 1.1)

 ## Тест различных моделей 
 
Попробуем различные модели

### 1. RandomForestRegressor

In [ ]:
model = RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1)
model.fit(X_train, y_train)

y_true = y_test
y_pred = model.predict(X_test)

print(f"Точность модели по метрике MAPE: {(mape(y_true, y_pred))*100:0.2f}%")
print(f"Точность модели по метрике MAPE если перевести в валюту: {(mape(np.exp(y_true) - 1, np.exp(y_pred) - 1))*100:0.2f}%")

### 2. LinearRegression

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

y_true = y_test
y_pred = model.predict(X_test)

print(f"Точность модели по метрике MAPE: {(mape(y_true, y_pred))*100:0.2f}%")
print(f"Точность модели по метрике MAPE если перевести в валюту: {(mape(np.exp(y_true) - 1, np.exp(y_pred) - 1))*100:0.2f}%")

### 3. ExtraTreeRegressor

In [ ]:
model = ExtraTreeRegressor(random_state = RANDOM_SEED)
model.fit(X_train, y_train)

y_true = y_test
y_pred = model.predict(X_test)

print(f"Точность модели по метрике MAPE: {(mape(y_true, y_pred))*100:0.2f}%")
print(f"Точность модели по метрике MAPE если перевести в валюту: {(mape(np.exp(y_true) - 1, np.exp(y_pred) - 1))*100:0.2f}%")

### 4. GradientBoostingRegressor

In [ ]:
# проверим градиентный бустинг 
model = GradientBoostingRegressor(n_estimators=250)
model.fit(X_train, y_train)

y_true = y_test
y_pred = model.predict(X_test)

print(f"Точность модели по метрике MAPE: {(mape(y_true, y_pred))*100:0.2f}%")
print(f"Точность модели по метрике MAPE если перевести в валюту: {(mape(np.exp(y_true) - 1, np.exp(y_pred) - 1))*100:0.2f}%")

### 5. Bagging на RandomForestRegressor

In [ ]:
# проверим BaggingRegressor вместе со случайным лесом на всем трейне
rf = RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1)
model = BaggingRegressor(rf, n_estimators=3, n_jobs=-1, random_state=RANDOM_SEED)
model.fit(X_train, y_train)

y_true = y_test
y_pred = model.predict(X_test)

print(f"Точность модели по метрике MAPE: {(mape(y_true, y_pred))*100:0.2f}%")
print(f"Точность модели по метрике MAPE если перевести в валюту: {(mape(np.exp(y_true) - 1, np.exp(y_pred) - 1))*100:0.2f}%")

Сравним с просто RandomForestRegressor

In [ ]:
model = RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1)
model.fit(X_train, y_train)

y_true = y_test
y_pred = model.predict(X_test)

print(f"Точность модели по метрике MAPE: {(mape(y_true, y_pred))*100:0.2f}%")
print(f"Точность модели по метрике MAPE если перевести в валюту: {(mape(np.exp(y_true) - 1, np.exp(y_pred) - 1))*100:0.2f}%")

### 6. Стеккинг на RandomForestRegressor и BaggingRegressor 

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

def compute_metric(clf, X_train=X_train, y_train=y_train, X_test=X_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    
    print(f"Точность модели по метрике MAPE: {(mape(y_test, y_test_pred))*100:0.2f}%")
    print(f"Точность модели по метрике MAPE если перевести в валюту: {(mape(np.exp(y_test) - 1, np.exp(y_test_pred) - 1))*100:0.2f}%")

def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
   
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.hstack([
        features_train for features_train, features_test in features
    ])

    stacked_features_test = np.hstack([
        features_test for features_train, features_test in features
    ])
    
    return stacked_features_train, stacked_features_test

def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    
    X_meta_train = np.zeros((len(y_train)), dtype=np.float32)

    splits = cv.split(X_train, y_train) 
    index = X_train.index
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train.loc[index[train_fold_index]], X_train.loc[index[predict_fold_index]]
        y_fold_train = y_train.loc[index[train_fold_index]]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict(X_test)
    
    return X_meta_train.reshape(len(X_meta_train), 1), X_meta_test.reshape(len(X_meta_test), 1)

In [ ]:
RANDOM_STATE = 42

classifiers = [ 
    RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1),
    BaggingRegressor(ExtraTreeRegressor(random_state=RANDOM_SEED), random_state=RANDOM_SEED)
]

meta_classifier = LinearRegression()

In [ ]:
from sklearn.base import clone

In [ ]:
stacked_features_train, stacked_features_test = generate_meta_features(classifiers, X_train, X_test, y_train, cv)

In [ ]:
compute_metric(clf=meta_classifier, X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test)

# 7. Итог:

1. Была написана программа по сбору данных с сайта auto.ru
2. Был подготовлен набор данных для обучения последующих алгоритмов

3. В результате работы над данными:
 - был произведен разведывательный анализ данных
 - созданы новые признаки
 - отбраны признаки для модели по значимости
 - проведена проверка различных моделей, в частности  был посторен стеккинг на RandomForestRegressor и BaggingRegressor

4. В финальной версии модели score - 10.85 (на валидации) и 19.34834 (Public Score на Kaggle)

# Submission

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_2_v{VERSION}.csv', index=False)
sample_submission.head(10)

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing